In [24]:
import cv2

# Input and output file paths
input_video_path = "video.mp4"
output_video_path = "output_video.mp4"
frame_divisor = 3

# Open the input video
cap = cv2.VideoCapture(input_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original FPS
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames in the input
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4

# Define the VideoWriter for output
out = cv2.VideoWriter(output_video_path, fourcc, fps / frame_divisor, (frame_width, frame_height))

frame_count = 0
output_frame_count = 0  # Counter for output frames

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Write every nth frame to the output video
    if frame_count % frame_divisor == 0:
        out.write(frame)
        output_frame_count += 1  # Increment output frame counter
    
    frame_count += 1

# Release resources
cap.release()
out.release()

# Print total frames
print(f"Total number of frames in the input video: {total_frames}")
print(f"Total number of frames in the output video: {output_frame_count}")
print(f"New video saved at {output_video_path}")


Total number of frames in the input video: 234
Total number of frames in the output video: 78
New video saved at output_video.mp4


In [17]:
import numpy as np
from PIL import Image

# Function to crop the image to 512x512
def crop_image(image, size=(1920, 1080)):
    width, height = image.size
    left = (width - size[0]) // 2
    top = (height - size[1]) // 2
    right = left + size[0]
    bottom = top + size[1]
    return image.crop((left, top, right, bottom))

# Load the image
image = Image.open('image.jpg')  # Replace 'image.jpg' with your image file
image = crop_image(image)  # Crop to 512x512

# Convert image to RGB if not already
image = image.convert('RGB')
image_array = np.array(image)

# Create the .h file and write the C-style array
with open('imageData.h', 'w') as file:
    file.write("char imageData[] = {\n")
    
    rows, cols, channels = image_array.shape
    for r in range(rows):
        for c in range(cols):
            # Write RGB values as a triplet for each pixel
            r_val, g_val, b_val = image_array[r, c]
            file.write(f"{r_val},{g_val},{b_val},")
        file.write("")  # Newline for readability
    file.seek(file.tell() - 2, 0)  # Remove the trailing comma
    file.write("\n};")

print("C-style RGB array saved to imageData.h")


C-style RGB array saved to imageData.h


In [1]:
import cv2
import numpy as np
from tqdm.notebook import tqdm

# Output header file name
header_file_name = "video_buffers.h"

# Video input file
video_file = "output_video.mp4"

# Frame dimensions
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080

# Open the video file
cap = cv2.VideoCapture(video_file)

if not cap.isOpened():
    raise IOError("Error: Cannot open video file.")

frame_count = 0
buffers = []

frame_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

for _ in range(frame_total):
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if no frames are left

    # Resize frame to desired dimensions
    frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))

    # Convert the frame to raw pixel data (flattened array)
    raw_data = frame.flatten()

    # Add the frame's pixel data to the list
    buffers.append(raw_data)
    frame_count += 1

cap.release()

# Write the header file
with open(header_file_name, "w") as header_file:
    header_file.write("#ifndef IMAGE_BUFFERS_H\n")
    header_file.write("#define IMAGE_BUFFERS_H\n\n")
    header_file.write(f"#define IMAGE_WIDTH  {IMAGE_WIDTH}\n")
    header_file.write(f"#define IMAGE_HEIGHT {IMAGE_HEIGHT}\n\n")

    for i, buffer in enumerate(buffers):
        header_file.write(f"unsigned char imageData{i + 1}[IMAGE_WIDTH * IMAGE_HEIGHT * 3] = {{\n")
        buffer_str = ", ".join(map(str, buffer))
        header_file.write(f"    {buffer_str}\n")
        header_file.write("};\n\n")

    header_file.write("// Array of pointers to image buffers for convenience\n")
    image_pointers = ", ".join([f"imageData{i + 1}" for i in tqdm(range(frame_count))])
    header_file.write(f"unsigned char *images[] = {{{image_pointers}}};\n\n")

    header_file.write("#endif // IMAGE_BUFFERS_H\n")

print(f"Header file '{header_file_name}' generated successfully with {frame_count} frames.")


Processing frames:   0%|          | 0/78 [00:00<?, ?it/s]

Header file 'video_buffers.h' generated successfully with 78 frames.
